*Serhii Vidernikov Y4459773H*

*Nikita Polyanskiy Y4441167L*


# Colorectal histology dataset with LeNet 5

# Preview: Neural Network examples with `tf.keras`

- `tf.keras` Software (http://keras.io/);

- Visual example of LeNet-5 (http://yann.lecun.com/exdb/lenet/) using MNIST digits dataset;

- Examples with common neural network topologies (https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/).

## Colorectal histology

Methods to diagnose colorectal using histology images (<https://zenodo.org/record/53169#.XGZemKwzbmG>, <https://www.tensorflow.org/datasets/catalog/colorectal_histology>)

In this case, the purpose is to classify the type of histology in a given image in the following categories:

- 0: TUMOR
- 1: STROMA
- 2: COMPLEX
- 3: LYMPHO
- 4: DEBRIS
- 5: MUCOSA
- 6: ADIPOSE
- 7: EMPTY

## Local instalation (option 1)

Install the following Python packages to run this notebook

`pip install pip -U`

`pip install tensorflow jupyter`

## Google Colab (option 2)

[Google Colab](https://colab.research.google.com/) is a research project created to help disseminate machine learning education and research. It's a `Jupyter notebook` environment that requires no setup to use and runs entirely in the cloud.

Colaboratory notebooks are stored in [Google Drive](https://drive.google.com) and can be shared just as you would with Google Docs or Sheets. Colaboratory is free to use.

For more information, see our [FAQ](https://research.google.com/colaboratory/faq.html).

### How install extra packages
Google Colab installs a series of basic packages if we need any additional package just install it.

In [ ]:
#!pip install scikit-learn -U
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 13.8 MB/s 


## Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay
from numpy import mean

from keras.optimizers import SGD, Nadam

from keras.utils import np_utils
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.utils import class_weight

import time

import albumentations as A

batch_size = 32 #Lote ok
nb_classes = 8  # values = 2 or 8
epochs = 100
optimizer = "adam"
generar = True
# Scaling input image to theses dimensions
img_rows, img_cols = 32,32

datagen = ImageDataGenerator(
        validation_split = 0.1,
        rotation_range=15, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
)

def format_example(image):
    image = tf.cast(image, tf.float32)
    # Normalize the pixel values
    image = image / 255.0
    # Resize the image
    image = tf.image.resize(image, (img_rows, img_cols))
    return image


def load_data(name="colorectal_histology"):
    train_ds = tfds.load(name, split=tfds.Split.TRAIN, batch_size=-1)
    train_ds['image'] = tf.map_fn(format_example, train_ds['image'], dtype=tf.float32)
    numpy_ds = tfds.as_numpy(train_ds)
    X, y = numpy_ds['image'], numpy_ds['label']

    return np.array(X), np.array(y)

def plot_symbols(X,y,n=15):
    index = np.random.randint(len(y), size=n)
    plt.figure(figsize=(25, 2))
    for i in np.arange(n):
        ax = plt.subplot(1,n,i+1)
        plt.imshow(X[index[i],:,:,:])
        plt.gray()
        ax.set_title(f'{y[index[i]]}-{index[i]}')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

  #
# Build an ANN structure - LeNet5
#
activation = "relu"
def cnn_model():
    #
    # Neural Network Structure
    #
    
    model = Sequential()
    
    model.add(layers.Conv2D(6, (5, 5)))
    model.add(layers.Activation(activation))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(16, (5, 5)))
    model.add(layers.Activation(activation))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        
    model.add(layers.Flatten())
    
    model.add(layers.Dense(120))
    model.add(layers.Activation(activation))
    
    model.add(layers.Dense(84))
    model.add(layers.Activation(activation))
    
    
    model.add(layers.Dense(nb_classes))
    model.add(layers.Activation('softmax'))

    return model


def show_both_images(a,b):
  f, axarr = plt.subplots(1,2)
  axarr[0].imshow(a, interpolation='nearest')
  axarr[1].imshow(b, interpolation='nearest')
  plt.show()

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

##################################################################################
# Main program

X, y = load_data()


print(X.shape, 'train samples')
print(img_rows,'x', img_cols, 'image size')
print(epochs,'epochs')

#plot_symbols(X, y)


if nb_classes==2:
  y[y>0] = 1

pd.DataFrame({'ncount':pd.Series(y).value_counts(), '% ncount':pd.Series(y).value_counts(normalize=True)*100})


#Codigo que se daba inicialmente
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# print(f'X_train {X_train.shape} X_test {X_test.shape}')
# print(f'y_train {y_train.shape} y_test {y_test.shape}')

#cross validation
i=0
skf = StratifiedKFold(n_splits = 10, random_state = 890, shuffle= True)
aucs = []
f1Results = []
start_time = time.time()

for train_index, test_index in skf.split(X,y):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  model = cnn_model()

  model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=[tf.keras.metrics.AUC()])

  y_Test2 = np_utils.to_categorical(y_test,nb_classes)
  y_Train2 = np_utils.to_categorical(y_train,nb_classes)

  patience = 10
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)


#-----------Generar imagenes---------------------
  if nb_classes==2 and generar==True:
    datagen.fit(X_train)
    X_tumor = []
    X_NoTumor = []
    Y_Tumor = []
    Y_NoTumor = []
    print(X_train.shape)

    for i in range(len(X_train)):
      if y_train[i] == 0:
        X_tumor.append(X_train[i])
        Y_Tumor.append(0)
      else:
        X_NoTumor.append(X_train[i])
        Y_NoTumor.append(1)

    print(len(X_tumor))
    print(len(X_NoTumor))

    X_tumor2 = np.array(X_tumor)
    c = 0
    for i in range(len(X_tumor2)*6):
      it = datagen.flow(X_tumor2,batch_size=1)
      img = next(it)[0].astype('uint8')
      X_tumor.append(img)
      Y_Tumor.append(0)
      c+=1

    print('contador: ',c)
    print(len(X_tumor))
    X_train = X_tumor + X_NoTumor
    Y_train = Y_Tumor + Y_NoTumor
    print(len(X_train))
    print(len(Y_train))

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_train, Y_train = unison_shuffled_copies(X_train,Y_train)

    Y_train = np_utils.to_categorical(Y_train,nb_classes)
    print(X_train.shape)
    print(Y_train.shape)

    model.fit(X_train,Y_train, batch_size=batch_size, epochs=epochs,validation_split=0.1, verbose=1, callbacks=[callback, PlotLossesKeras()])

  elif nb_classes==8 and generar==True:
#----------------------- Usando Albumentation ----------------------- 
    print("Generando con albumentation...")
    XT = []
    YT = []

    transform = A.Compose([A.Rotate(limit=45),A.RandomBrightnessContrast(p=1.0,brightness_limit=0.6)])
    print(len(XT))
    print(len(YT))


    for i in range(len(X_train)):
      XT.append(X_train[i])
      YT.append(y_Train2[i])
    print('Inicializados')
    print(len(XT))
    print(len(YT))
    l = len(XT)

    c=0
    for i in range(l):
      transformed = transform(image=XT[i])
      transformed_img=transformed["image"]
      #show_both_images(X_train[i],transformed_img)
      XT.append(transformed_img)
      YT.append(YT[i])
      c+=1
      #print("Imagenes: ",c)


    print("Imagenes: ",c)

    print("Despues de generar")
    print(len(XT))
    print(len(YT))

#Imprimir TODO!

    XT = np.array(XT)
    YT = np.array(YT)

    XT,YT=unison_shuffled_copies(XT,YT)
    model.fit(XT,YT, batch_size=batch_size, epochs=epochs,validation_split=0.1, verbose=1, callbacks=[callback])

  else:
    model.fit(X_train,y_Train2, batch_size=batch_size, epochs=epochs,validation_split=0.1, verbose=1, callbacks=[callback])


  #print(model.summary())
  loss, acc = model.evaluate(X_test, y_Test2, batch_size=batch_size)
  print(f'loss: {loss:.2f} acc: {acc:.2f}')

  y_scores = model.predict(X_test) # Confidence prediction per class
  y_pred = y_scores.argmax(axis=1) # Select classes with most confidence prediction

  if nb_classes ==  2:
    AUC = metrics.roc_auc_score(y_test, np.round(y_scores[:,1],2))
    aucs.append(round(AUC,4))
    print("################################################################")
    print(f'AUC {metrics.roc_auc_score(y_test, np.round(y_scores[:,1],2)):.4f} ')
    print("################################################################")
    
    fpr, tpr, _ = metrics.roc_curve(y_test, y_scores[:,1],pos_label=1)
    #print(fpr, tpr)

    roc_auc = metrics.auc(fpr, tpr)
    #Imprimir curva Roc
    #roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=AUC).plot()
    
  else:
    print(f'AUC {metrics.roc_auc_score(y_test, y_scores, multi_class = "ovr"):.4f} ')
    AUC = metrics.roc_auc_score(y_test, y_scores, multi_class = "ovr")
    aucs.append(round(AUC,4))



  # Output prediction for the first five X_test images
  model.predict(X_test[:5], batch_size=batch_size)

  print('Predictions', pd.Series(y_pred).value_counts(),'\n')

  print('Confusion matrix')
  print(metrics.confusion_matrix(y_test,y_pred),'\n')

  target_names = ['TUMOR', 'HEALTHY'] if nb_classes ==  2 else ['TUMOR','STROMA','COMPLEX','LYMPHO','DEBRIS','MUCOSA','ADIPOSE','EMPTY']

  #print(metrics.classification_report(y_test, y_pred, target_names=target_names))
  result = metrics.classification_report(y_test, y_pred, target_names=target_names)
  f1 = round(metrics.f1_score(y_test, y_pred,average='macro'),4)
  print(result)
  print("F1: ",f1)
  f1Results.append(f1)
  
  # if( i == 0):
  #   masker = shap.maskers.Image("blur("+ str(img_rows) +"," + str(img_cols) + ")", X_train[0].shape)
  #   explainer = shap.Explainer(model, masker, output_names=target_names)
  #   shap_values = explainer(X_test[:8], outputs=shap.Explanation.argsort.flip[:8])
  #   shap_values
  #   shap.image_plot(shap_values)
  # i = i + 1

print('Optimizador: ',optimizer)
print('Activacion: ',activation)
print('Train samples: ',X.shape)
print('Image size: ',img_rows,'x', img_cols)
print('Batch size: ', batch_size)
print('Epochs: ',epochs)
print('Patience: ',patience)
print('AUCs: ',aucs)
print('F1 Scores: ',f1Results)
print('Media de aucs:', mean(aucs))
print('Tiempo: ',time.time()-start_time)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/colorectal_histology/2.0.0.incompleteSE7XLE/colorectal_histology-train.tfrecor…

Dataset colorectal_histology downloaded and prepared to ~/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.
(5000, 32, 32, 3) train samples
32 x 32 image size
100 epochs
Generando con albumentation...
0
0
Inicializados
4500
4500
Imagenes:  4500
Despues de generar
9000
9000
Epoch 1/100
254/254 [==============================] - 11s 7ms/step - loss: 0.3182 - auc: 0.8029 - val_loss: 0.2643 - val_auc: 0.8780
Epoch 2/100
254/254 [==============================] - 1s 5ms/step - loss: 0.2626 - auc: 0.8740 - val_loss: 0.2383 - val_auc: 0.9039
Epoch 3/100
254/254 [==============================] - 1s 5ms/step - loss: 0.2459 - auc: 0.8921 - val_loss: 0.2570 - val_auc: 0.8787
Epoch 4/100
254/254 [==============================] - 1s 5ms/step - loss: 0.2352 - auc: 0.9028 - val_loss: 0.2152 - val_auc: 0.9216
Epoch 5/100
254/254 [==============================] - 1s 5ms/step - loss: 0.2181 - auc: 0.9186 - val_loss: 0.2096 - val_auc: 0.9248
Epoch 6/100
254/254 [==